In [4]:
import pandas as pd
df = pd.read_csv('datasets/POS_TRANSACTIONS_2018 .csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459258 entries, 0 to 459257
Data columns (total 5 columns):
Location            459258 non-null int64
Transaction_Id      459258 non-null int64
Transaction_Date    459258 non-null object
Product_Name        459258 non-null object
Quantity            459258 non-null int64
dtypes: int64(3), object(2)
memory usage: 17.5+ MB


In [5]:
print('----Location----')
print(df['Location'].value_counts())
print('-------Transaction_Id-------')
print(df['Transaction_Id'].value_counts())
print('----Transaction_Date----')
print(df['Transaction_Date'].value_counts())
print('----Product_Name----')
print(df['Product_Name'].value_counts())
print('----Quantity----')
print(df['Quantity'].value_counts())

----Location----
7     46570
3     46381
6     46299
2     46100
5     46032
8     45993
1     45896
10    45516
4     45354
9     45117
Name: Location, dtype: int64
-------Transaction_Id-------
205484     170
79610      164
362087     148
816536     135
1211036    126
34322      123
450677     117
75389      107
1143932     95
205490      94
824726      86
1014407     83
873266      81
297173      80
1068674     75
258056      75
967577      74
427835      73
743966      69
205496      69
712436      66
1140599     63
1131095     62
513455      61
246527      60
79655       60
1118885     59
194294      59
106274      59
1172354     58
          ... 
1095557      1
34691        1
561026       1
40832        1
999284       1
686045       1
1038185      1
495458       1
935771       1
933716       1
919379       1
925520       1
585614       1
1056656      1
1058711      1
1152929      1
114596       1
642986       1
116651       1
1177517      1
651182       1
69554        1
77750     

In [14]:
transactionsTest = df.groupby(['Transaction_Id', 'Transaction_Date'])['Product_Name'].apply(list)
print(transactionsTest.head(10))

Transaction_Id  Transaction_Date
12359           12/23/2017                                            [Exercise book]
12362           12/23/2017                 [Mini Stationery Set, Mini Stationery Set]
12365           12/23/2017                                               [Flash Card]
12371           12/23/2017                                             [Drink bottle]
12380           12/23/2017                                                [DVD media]
12383           12/23/2017                 [Mini Stationery Set, Mini Stationery Set]
12386           12/23/2017                 [Mini Stationery Set, Mini Stationery Set]
12392           12/23/2017                     [Certificate Frame, Sketching Markers]
12401           12/23/2017          [Exercise book, Exercise book, Lanyards, Sketc...
12404           12/23/2017                                            [Exercise book]
Name: Product_Name, dtype: object


In [15]:
transactions = df.groupby(['Transaction_Id'])['Product_Name'].apply(list)
print(transactions.head(10))

Transaction_Id
12359                                      [Exercise book]
12362           [Mini Stationery Set, Mini Stationery Set]
12365                                         [Flash Card]
12371                                       [Drink bottle]
12380                                          [DVD media]
12383           [Mini Stationery Set, Mini Stationery Set]
12386           [Mini Stationery Set, Mini Stationery Set]
12392               [Certificate Frame, Sketching Markers]
12401    [Exercise book, Exercise book, Lanyards, Sketc...
12404                                      [Exercise book]
Name: Product_Name, dtype: object


In [63]:
from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.03, min_confidence=0.1))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'DVD media'}), support=0.146885, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'DVD media'}), confidence=0.146885, lift=1.0)]), RelationRecord(items=frozenset({'Exercise book'}), support=0.171005, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Exercise book'}), confidence=0.171005, lift=1.0)]), RelationRecord(items=frozenset({'Flash Card'}), support=0.160425, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Flash Card'}), confidence=0.160425, lift=1.0)]), RelationRecord(items=frozenset({'Lanyards'}), support=0.134925, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Lanyards'}), confidence=0.134925, lift=1.0)]), RelationRecord(items=frozenset({'Sketching Markers'}), support=0.241305, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Sketching Markers'}), confidence=0.241305, lift=1.0

In [64]:
def convert_apriori_results_to_pandas_df(results):
    rules = []

    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
            rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])
result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.head(20))

            Left_side         Right_side   Support  Confidence      Lift
0                              DVD media  0.146885    0.146885  1.000000
1                          Exercise book  0.171005    0.171005  1.000000
2                             Flash Card  0.160425    0.160425  1.000000
3                               Lanyards  0.134925    0.134925  1.000000
4                      Sketching Markers  0.241305    0.241305  1.000000
5                             Wristbands  0.143575    0.143575  1.000000
6           DVD media      Exercise book  0.043660    0.297239  1.738191
7       Exercise book          DVD media  0.043660    0.255314  1.738191
8           DVD media         Flash Card  0.032080    0.218402  1.361397
9          Flash Card          DVD media  0.032080    0.199969  1.361397
10          DVD media  Sketching Markers  0.036335    0.247370  1.025136
11  Sketching Markers          DVD media  0.036335    0.150577  1.025136
12      Exercise book         Flash Card  0.039780 

In [65]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(20))

            Left_side         Right_side   Support  Confidence      Lift
6           DVD media      Exercise book  0.043660    0.297239  1.738191
7       Exercise book          DVD media  0.043660    0.255314  1.738191
12      Exercise book         Flash Card  0.039780    0.232625  1.450053
13         Flash Card      Exercise book  0.039780    0.247966  1.450053
15           Lanyards      Exercise book  0.033015    0.244691  1.430903
14      Exercise book           Lanyards  0.033015    0.193065  1.430903
8           DVD media         Flash Card  0.032080    0.218402  1.361397
9          Flash Card          DVD media  0.032080    0.199969  1.361397
11  Sketching Markers          DVD media  0.036335    0.150577  1.025136
10          DVD media  Sketching Markers  0.036335    0.247370  1.025136
1                          Exercise book  0.171005    0.171005  1.000000
0                              DVD media  0.146885    0.146885  1.000000
5                             Wristbands  0.143575 

In [66]:
new_df = result_df[result_df['Left_side'] == 'Exercise book']
print(new_df.head(20))

        Left_side         Right_side   Support  Confidence      Lift
7   Exercise book          DVD media  0.043660    0.255314  1.738191
12  Exercise book         Flash Card  0.039780    0.232625  1.450053
14  Exercise book           Lanyards  0.033015    0.193065  1.430903
16  Exercise book  Sketching Markers  0.040535    0.237040  0.982325
